# My dataset with VGG16


### 데이터준비

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders   # 이미지 데이터 분리시 사용

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
# 데이터 준비
# !unzip -qq "/content/animal.zip" -d "/content/animal"

# !unzip -qq "/content/animal.zip"    # animal.zip안에 animal_data 폴더가 존재해서 그냥 압축 해제

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 입력 폴더 (각 클래스별로 폴더가 존재해야 함)
input_folder = '/content/drive/MyDrive/KDT뱀조/snake'

# 출력 폴더 (여기에 train, val, test 폴더가 생성됨)
output_folder = '/content'

# 비율로 분할: train 70%, validation 20%, test 10%
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.7, .2, .1))

# splitfolders.fixed("input_folder", output="output", seed=1337, fixed=(100, 100))

Copying files: 1972 files [01:48, 18.16 files/s]


In [ ]:
train = ImageDataGenerator(rescale= 1./255)
val = ImageDataGenerator(rescale= 1./255)
test = ImageDataGenerator(rescale= 1./255)

In [ ]:
train_dataset = train.flow_from_directory("/content/train",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

val_dataset = val.flow_from_directory("/content/val",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

test_dataset = test.flow_from_directory("/content/test",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

Found 1348 images belonging to 7 classes.
Found 388 images belonging to 7 classes.
Found 199 images belonging to 7 classes.


In [ ]:
train_dataset.class_indices

{'까치살모사': 0,
 '누룩뱀': 1,
 '능구렁이': 2,
 '물뱀': 3,
 '쇠살모사': 4,
 '유혈목이(꽃뱀)': 5,
 '황구렁이': 6}

In [ ]:
train_dataset.classes

array([0, 0, 0, ..., 6, 6, 6], dtype=int32)

In [ ]:
print('img shape:', train_dataset[0][0][0].shape) # 1번째 배치의 이미지들의 첫번째 이미지
print('label:', train_dataset[0][1][0]) # 1번째 배치의 레이블들의 첫번째 레이블

img shape: (224, 224, 3)
label: 1.0


In [ ]:
class_weight = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0}
# Adjust weights according to the number of samples in each class
model.fit(train_dataset, validation_data=val_dataset, epochs=10, class_weight=class_weight)

Epoch 1/10
43/43 [==============================] - 18s 423ms/step - loss: nan - accuracy: 0.0809 - val_loss: nan - val_accuracy: 0.0799
Epoch 2/10
43/43 [==============================] - 19s 438ms/step - loss: nan - accuracy: 0.0809 - val_loss: nan - val_accuracy: 0.0799
Epoch 3/10
13/43 [========>.....................] - ETA: 9s - loss: nan - accuracy: 0.0817

KeyboardInterrupt: 

# Small CNN

In [ ]:
# 우리가 만든 vgg16의 정확도가 나오지 않는 것을 확인하고 작은 모델로 다시 테스트하기 위함

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 모델 생성
model = Sequential([
    # 첫 번째 컨볼루션 레이어
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    # 두 번째 컨볼루션 레이어
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # Flatten 후 Fully Connected 레이어
    Flatten(),
    Dense(512, activation='relu'),
    # Dropout을 추가하여 과적합 방지
    Dropout(0.5),
    # 출력 레이어
    Dense(4, activation='softmax') # 뱀 종류...!
])

In [ ]:
# 모델 컴파일, 훈련, 평가하고 예측을 만듭니다.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 최적화를 위한 설정 구간입니다.
modelpath="./custom_vgg16.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(train_dataset, validation_data= val_dataset, epochs=20, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/20
43/43 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0809
Epoch 1: val_loss did not improve from inf
43/43 [==============================] - 25s 442ms/step - loss: nan - accuracy: 0.0809 - val_loss: nan - val_accuracy: 0.0799
Epoch 2/20
43/43 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0809
Epoch 2: val_loss did not improve from inf
43/43 [==============================] - 18s 416ms/step - loss: nan - accuracy: 0.0809 - val_loss: nan - val_accuracy: 0.0799
Epoch 3/20
43/43 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0809
Epoch 3: val_loss did not improve from inf
43/43 [==============================] - 18s 421ms/step - loss: nan - accuracy: 0.0809 - val_loss: nan - val_accuracy: 0.0799
Epoch 4/20
43/43 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0809
Epoch 4: val_loss did not improve from inf
43/43 [==============================] - 18s 419ms/step - loss: nan - accuracy: 0.0809

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.3f}")

7/7 [==============================] - 2s 264ms/step - loss: nan - accuracy: 0.0854
Test Accuracy: 0.085


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset = train_datagen.flow_from_directory(
    "/content/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)


Found 1348 images belonging to 7 classes.


## Pre-trained VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from functools import partial

In [ ]:
base_model = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# 사전 학습된 레이어는 학습되지 않도록 설정
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# 모델 구조 커스터마이징
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # 뱀 종류..!

# 새로운 모델 정의
model = Model(inputs= base_model.input, outputs= predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# 모델 컴파일, 훈련, 평가하고 예측을 만듭니다.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 최적화를 위한 설정 구간입니다.
modelpath="./pretrained_vgg16.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(train_dataset, validation_data= val_dataset, epochs=50, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/50
8/8 [==============================] - ETA: 0s - loss: 3.9268 - accuracy: 0.4647
Epoch 1: val_loss improved from inf to 5.24062, saving model to ./pretrained_vgg16.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 19s 1s/step - loss: 3.9268 - accuracy: 0.4647 - val_loss: 5.2406 - val_accuracy: 0.5714
Epoch 2/50
8/8 [==============================] - ETA: 0s - loss: 2.3424 - accuracy: 0.6888
Epoch 2: val_loss improved from 5.24062 to 1.71299, saving model to ./pretrained_vgg16.hdf5
8/8 [==============================] - 5s 646ms/step - loss: 2.3424 - accuracy: 0.6888 - val_loss: 1.7130 - val_accuracy: 0.5714
Epoch 3/50
8/8 [==============================] - ETA: 0s - loss: 1.4001 - accuracy: 0.7344
Epoch 3: val_loss did not improve from 1.71299
8/8 [==============================] - 4s 476ms/step - loss: 1.4001 - accuracy: 0.7344 - val_loss: 1.8689 - val_accuracy: 0.5857
Epoch 4/50
8/8 [==============================] - ETA: 0s - loss: 0.8175 - accuracy: 0.8216
Epoch 4: val_loss improved from 1.71299 to 1.20826, saving model to ./pretrained_vgg16.hdf5
8/8 [==============================] - 4s 507ms/step - loss: 0.8175 - accuracy: 0.8216 - val_loss: 1.2083 - 

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.3f}")

2/2 [==============================] - 2s 2s/step - loss: 0.9401 - accuracy: 0.7297
Test Accuracy: 0.730


## Pretrained ResNet50

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [ ]:
# resnet50에 맞는 이미지 전처리를 위해 preprocess_input 함수 사용
# 모델에 맞는 전처리가 적용되지 않으면 정확도가 이상하게 나올 수 있다는 것 확인

train = ImageDataGenerator(preprocessing_function= preprocess_input)
val = ImageDataGenerator(preprocessing_function= preprocess_input)
test = ImageDataGenerator(preprocessing_function= preprocess_input)

In [ ]:
train_dataset = train.flow_from_directory("/content/train",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

val_dataset = val.flow_from_directory("/content/val",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

test_dataset = test.flow_from_directory("/content/test",
                                          target_size= (224, 224),
                                          batch_size= 32,
                                          class_mode= 'sparse')

Found 241 images belonging to 3 classes.
Found 70 images belonging to 3 classes.
Found 37 images belonging to 3 classes.


In [ ]:
base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
# 사전 학습된 레이어는 학습되지 않도록 설정
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
# 모델 구조 커스터마이징
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x) # 뱀 종류...!

# 새로운 모델 정의
model = Model(inputs= base_model.input, outputs= predictions)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
# 모델 컴파일, 훈련, 평가하고 예측을 만듭니다.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 최적화를 위한 설정 구간입니다.
modelpath="./pretrained_resnet50.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(train_dataset, validation_data= val_dataset, epochs=50, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/50
8/8 [==============================] - ETA: 0s - loss: 23.0068 - accuracy: 0.5311
Epoch 1: val_loss improved from inf to 7.82847, saving model to ./pretrained_resnet50.hdf5
8/8 [==============================] - 15s 1s/step - loss: 23.0068 - accuracy: 0.5311 - val_loss: 7.8285 - val_accuracy: 0.7143
Epoch 2/50
8/8 [==============================] - ETA: 0s - loss: 6.7428 - accuracy: 0.8382
Epoch 2: val_loss did not improve from 7.82847
8/8 [==============================] - 4s 578ms/step - loss: 6.7428 - accuracy: 0.8382 - val_loss: 20.9997 - val_accuracy: 0.6286
Epoch 3/50
8/8 [==============================] - ETA: 0s - loss: 5.7646 - accuracy: 0.8755
Epoch 3: val_loss did not improve from 7.82847
8/8 [==============================] - 4s 511ms/step - loss: 5.7646 - accuracy: 0.8755 - val_loss: 8.0196 - val_accuracy: 0.7429
Epoch 4/50
8/8 [==============================] - ETA: 0s - loss: 1.1406 - accuracy: 0.9502
Epoch 4: val_loss improved from 7.82847 to 6.83932, saving 

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.3f}")

2/2 [==============================] - 2s 1s/step - loss: 7.8803 - accuracy: 0.7838
Test Accuracy: 0.784
